In [ ]:
from google import genai
from google.genai import types
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

client = genai.Client(api_key='API_KEY_HERE')

texts = [
    "What is the meaning of life?",
    "What is the purpose of existence?",
    "How do I bake a cake?"]

result = [
    np.array(e.values) for e in client.models.embed_content(
        model="gemini-embedding-001",
        contents=texts,
        config=types.EmbedContentConfig(task_type="SEMANTIC_SIMILARITY")).embeddings
]

# Calculate cosine similarity. Higher scores = greater semantic similarity.

embeddings_matrix = np.array(result)
similarity_matrix = cosine_similarity(embeddings_matrix)

for i, text1 in enumerate(texts):
    for j in range(i + 1, len(texts)):
        text2 = texts[j]
        similarity = similarity_matrix[i, j]
        print(f"Similarity between '{text1}' and '{text2}': {similarity:.4f}")

Similarity between 'What is the meaning of life?' and 'What is the purpose of existence?': 0.9417
Similarity between 'What is the meaning of life?' and 'How do I bake a cake?': 0.7676
Similarity between 'What is the purpose of existence?' and 'How do I bake a cake?': 0.7471


In [13]:
EMBEDDING_MODEL = "gemini-embedding-001"

In [14]:
import json

with open("shop_data.json", "r", encoding="utf-8") as f:
    products = json.load(f)
    
products = products['products']

In [47]:
def build_embedding_text(p: dict) -> str:
    # Lấy danh sách options nếu có
    options = p.get('flower_details', {}).get('options', [])
    
    if options:
        prices = [opt.get('price') for opt in options if isinstance(opt.get('price'), (int, float))]
        if prices:
            min_price = min(prices)
            max_price = max(prices)
            price_text = f"Price range: from {min_price} to {max_price} (optional)."
        else:
            price_text = ""
    else:
        price_text = ""

    # Tạo mô tả embedding
    return f"""
ID: {p['product_id']}
Name: {p['name']}
Description: {p.get('description', '')}
Type: {p.get('flower_details', {}).get('flower_type', '')}
Color: {', '.join(p.get('flower_details', {}).get('color', []))}
Occasion: {', '.join(p.get('flower_details', {}).get('occasion', []))}
Options: {p.get('flower_details', {}).get('options', [])}
{price_text}
""".strip()


In [48]:
# Chọn text để encode (tùy chỉnh theo nhu cầu)
texts = [
    build_embedding_text(p) for p in products
]

In [49]:
texts

["ID: B1\nName: Red Rose Bouquet\nDescription: A luxurious bouquet of fresh red roses, hand-tied with lush green foliage. Perfect for weddings, anniversaries, or romantic gestures, these premium roses convey love, passion, and elegance. Each stem is carefully selected for vibrant color, long-lasting freshness, and soft velvety petals.\nType: Roses\nColor: Pink Flowers\nOccasion: Wedding\nOptions: [{'name': 'original', 'price': 19.99, 'stems': 5, 'stock': 3}, {'name': 'deluxe', 'price': 29.99, 'stems': 10, 'stock': 0}, {'name': 'grand', 'price': 39.99, 'stems': 15, 'stock': 0}]\nPrice range: from 19.99 to 39.99 (optional).",
 "ID: B2\nName: Farmer's Choice Peonies\nDescription: A charming assortment of seasonal peonies sourced directly from local farmers. Featuring large, fragrant blooms in soft white shades, these flowers symbolize prosperity, good fortune, and honor. Ideal for celebrations, thank-you gifts, or welcoming a new baby, each bouquet is artfully arranged for maximum freshne

In [50]:
def embed_texts_google(texts, batch_size=1):
    all_vecs = []
    for i in range(0, len(texts), batch_size):
        chunk = texts[i:i+batch_size]
        resp = client.models.embed_content(
            model=EMBEDDING_MODEL,
            contents=chunk,
            config=types.EmbedContentConfig(task_type="SEMANTIC_SIMILARITY"),
        )
        vecs = [np.array(e.values, dtype=np.float32) for e in resp.embeddings]
        all_vecs.extend(vecs)
    mat = np.vstack(all_vecs)
    # L2-normalize for fast cosine similarity
    mat /= (np.linalg.norm(mat, axis=1, keepdims=True) + 1e-12)
    return mat

In [51]:
EMB_CACHE = "product_embeddings.npy"

In [52]:
import os

In [53]:
if os.path.exists(EMB_CACHE):
    product_embs = np.load(EMB_CACHE)
else:
    product_embs = embed_texts_google(texts)
    np.save(EMB_CACHE, product_embs)

In [67]:
# ----- 4) Semantic search over products -----
def semantic_search(query: str, top_k: int = 3):
    # embed query
    q_resp = client.models.embed_content(
        model=EMBEDDING_MODEL,
        contents=[query],
        config=types.EmbedContentConfig(task_type="SEMANTIC_SIMILARITY"),
    )
    q = np.array(q_resp.embeddings[0].values, dtype=np.float32)
    q /= (np.linalg.norm(q) + 1e-12)

    sims = cosine_similarity(q.reshape(1, -1), product_embs)[0]  # shape (N,)
    idx = np.argsort(-sims)[:top_k]
    indices = [texts[i].splitlines()[0].split()[1] for i in idx]
    return [product for product in products if product.get('product_id','') in indices]

In [68]:
question = "Please give me the price of the Rose with grand version"

In [69]:
semantic_search(question, top_k=3)

[{'product_id': 'B1',
  'type': 'flower',
  'name': 'Red Rose Bouquet',
  'price': 19.99,
  'stock': 12,
  'available': True,
  'description': 'A luxurious bouquet of fresh red roses, hand-tied with lush green foliage. Perfect for weddings, anniversaries, or romantic gestures, these premium roses convey love, passion, and elegance. Each stem is carefully selected for vibrant color, long-lasting freshness, and soft velvety petals.',
  'image_url': ['/src/assets/demo_1.png',
   '/src/assets/demo.png',
   '/src/assets/demo_2.png',
   '/src/assets/demo_3.png'],
  'flower_details': {'occasion': ['Wedding'],
   'color': ['Pink Flowers'],
   'flower_type': 'Roses',
   'options': [{'name': 'original', 'price': 19.99, 'stems': 5, 'stock': 3},
    {'name': 'deluxe', 'price': 29.99, 'stems': 10, 'stock': 0},
    {'name': 'grand', 'price': 39.99, 'stems': 15, 'stock': 0}]}},
 {'product_id': 'B3',
  'type': 'flower',
  'name': 'Peony Bouquet',
  'price': 40,
  'stock': 18,
  'available': True,
  'd

In [ ]:
def user_prompt_handler(user_message: str, k: int = 3) -> str:
    # Get relevant products
    retrieved_products = semantic_search(user_message, k)

    def normalize_product(p: dict):
        """Normalize both flower and vase products into a clean context."""
        if p.get("type") == "vase":
            return {
                "id": p.get("product_id"),
                "type": "vase",
                "name": p.get("name"),
                "description": p.get("description", ""),
                "price": p.get("price"),
                "available": bool(p.get("available", False)),
            }
        else:  # assume flower by default
            opts = []
            for o in p.get("flower_details", {}).get("options", []):
                opts.append({
                    "name": o.get("name"),
                    "price": o.get("price"),
                    "stems": o.get("stems"),
                    "available": bool(o.get("stock", 0) and o.get("stock", 0) > 0),
                })
            return {
                "id": p.get("product_id"),
                "type": "flower",
                "name": p.get("name"),
                "description": p.get("description", ""),
                "flower_type": p.get("flower_details", {}).get("flower_type", ""),
                "color": p.get("flower_details", {}).get("color", []),
                "occasion": p.get("flower_details", {}).get("occasion", []),
                "options": opts,
            }

    ctx_items = [normalize_product(p) for p in retrieved_products]
    context_text = json.dumps(ctx_items, ensure_ascii=False, indent=2)

    prompt = f"""
You are a friendly assistant helping customers choose products.  
Always start your answer with a short friendly introduction sentence before listing details.  
Answer based strictly on the [Shop Data] below.

[Shop Data]
{context_text}

[User Question]
{user_message}

[Instructions]
1. Start every response with a friendly, natural intro (e.g., "Here are some flowers you might like:" or "This vase could be a perfect match:").
2. Use facts ONLY from [Shop Data]. If missing, reply exactly: Data not found.
3. Prices:
   - Flowers: If a user asks about an option (original/deluxe/grand), provide the price if it exists.
   - Vases: Provide the single listed price if available.
4. Availability:
   - You may say "currently out of stock" if "available" is false.
   - Never mention or infer stock numbers.
5. Alternatives:
   - Flowers: If requested option is unavailable, suggest the nearest available tier (original ↔ deluxe ↔ grand).
   - Vases: If not available, simply say it’s out of stock.
6. Formatting:
   - Be concise, friendly, and natural.
   - List multiple products/options with "-" bullets.
   - Include product and option names with their prices.
   - Do NOT output JSON or code.
   - Do NOT include reasoning steps.

[Examples]
Q: What's the price of the Red Rose Bouquet grand version?  
A: Here’s what I found: The grand Red Rose Bouquet is 39.99, but it’s currently out of stock. A similar option is "deluxe" at 29.99.

Q: Do you have purple orchids for Women's Day?  
A: Here are some flowers that are suitable for Women’s Day:  
- Orchid Elegance — "original" 35.0, "deluxe" 55.0, "grand" 79.0.

Q: What’s the price of the Montecito Vase?  
A: This vase could be a perfect match: The Montecito Vase is available at 19.

Q: What if something isn’t in the data?  
A: Data not found.
"""
    return prompt

In [72]:
import requests
import json
import os
from dotenv import load_dotenv

In [73]:
# Load environment variables from .env file
load_dotenv()

# Check if API key is set
api_key = os.getenv('OPENROUTER_API_KEY_LLMA')
if not api_key:
    print("Error: OPENROUTER_API_KEY environment variable not set")
    exit(1)

In [74]:
def llm_response(user_message: str, k: int = 3):

    response = requests.post(
        url="https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json",
            # "HTTP-Referer": "https://your-site-url.com",  # Optional
            # "X-Title": "Your Site Name",  # Optional
        },
        data=json.dumps({
            "model": "deepseek/deepseek-r1-distill-llama-70b:free",
            "messages": [
                {
                    "role": "user",
                    "content": f"{user_prompt_handler(user_message, k)}"
                }
            ],
        })
    )
    return response

In [75]:
user_message_price = "Please give me the price of the Rose with grand version"

In [81]:
user_message_price = 'give me some vase of the shop'

In [82]:
semantic_search(user_message_price)

[{'product_id': 'V1',
  'type': 'vase',
  'name': 'Montecito Vase',
  'price': 19,
  'stock': 12,
  'available': True,
  'description': 'A sleek and modern glass vase with a timeless cylindrical design. Perfect for showcasing medium to large floral arrangements, this vase enhances the beauty of any bouquet while adding a touch of sophistication to your home décor.',
  'image_url': ['/src/assets/demo_3.png']},
 {'product_id': 'V2',
  'type': 'vase',
  'name': 'Montecitoss Vase',
  'price': 40,
  'stock': 0,
  'available': True,
  'description': 'A tall, elegant vase with a contemporary silhouette and premium finish. Designed to complement both classic and modern interiors, it’s ideal for displaying long-stemmed flowers or as a standalone statement piece.',
  'image_url': ['/src/assets/demo_4.png']},
 {'product_id': 'B5',
  'type': 'flower',
  'name': 'Rainbow Tulip Mix',
  'price': 22.5,
  'stock': 20,
  'available': True,
  'description': "Mixed-color tulips arranged in a clean, modern

In [83]:
user_prompt_handler(user_message_price, 3)

'\nYou are an friendly assistant that answers user questions based strictly on the provided data.\n\n[Shop Data]:\n{"product_id": "V1", "type": "vase", "name": "Montecito Vase", "price": 19, "stock": 12, "available": true, "description": "A sleek and modern glass vase with a timeless cylindrical design. Perfect for showcasing medium to large floral arrangements, this vase enhances the beauty of any bouquet while adding a touch of sophistication to your home décor.", "image_url": ["/src/assets/demo_3.png"]}\n{"product_id": "V2", "type": "vase", "name": "Montecitoss Vase", "price": 40, "stock": 0, "available": true, "description": "A tall, elegant vase with a contemporary silhouette and premium finish. Designed to complement both classic and modern interiors, it’s ideal for displaying long-stemmed flowers or as a standalone statement piece.", "image_url": ["/src/assets/demo_4.png"]}\n{"product_id": "B5", "type": "flower", "name": "Rainbow Tulip Mix", "price": 22.5, "stock": 20, "availabl

In [80]:
response = llm_response(user_message_price)

# Check response deep-seek
if response.status_code == 200:
    try:
        data = response.json()
        print("Response:", data['choices'][0]['message']['content'])
    except (KeyError, ValueError) as e:
        print(f"Error parsing response: {e}")
else:
    print(f"Error: {response.status_code} - {response.text}")

Response: The price of the Red Rose Bouquet grand version is 39.99.
